In [2]:
import numpy as np
import copy
import time

In [1]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [31]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [40]:
std0 = torch.eye(3, dtype=torch.float64) * 0.3
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(3, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", torch.zeros(3, dtype=torch.float64))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

In [41]:
X, Y, beta = generate_stdp_dataset(3, 10, 0, 1)

In [42]:
print(X)
print(Y)
print(beta)

[[3.84818176e-02 1.73799500e-01 7.54426464e-02]
 [4.16334038e-01 9.83568448e-02 2.25656680e-01]
 [3.67426058e-01 2.71284899e-01 4.28881190e-01]
 [1.00710083e-01 8.77833534e-02 1.22729504e-01]
 [2.21131705e-01 7.81724563e-02 5.56578332e-01]
 [5.08464293e-01 4.51489138e-01 4.20074767e-04]
 [4.58421744e-01 4.48489260e-01 2.88152409e-02]
 [3.51971309e-01 4.00837855e-01 2.65774892e-01]
 [8.78063644e-02 2.34246851e-01 2.83559913e-01]
 [1.63264992e-01 2.30884878e-01 5.27790893e-01]
 [6.48521993e-02 4.45950512e-01 6.45769750e-02]]
[ 0.19498061  0.18087539 -0.13223411 -0.57481897  0.27271433 -0.34902969
  0.35609523  0.11075488 -0.4143449   0.10199898  0.25300824]
[0.25940649 0.64420313 0.20139894]


In [43]:
np.std(Y_train)

0.30966043092725287

In [44]:
def train_SVI(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        svi.step(D_hat)
    
    breakpoint()
    mu_q = pyro.param("mu_q")
    return mu_q

In [45]:

print(y_hat)
print(y_bottom)
conformal_set = []
step_size = 0.1
start = time.time()
while y_hat >= y_bottom:
    pyro.clear_param_store()
    # Create D_hat
    D_hat = list(zip(X_train[:-1], Y_train))
    D_hat.append((X_train[-1], y_hat))
    
    # Train SVI
    mu_q = train_SVI(D_hat, 10)
    
    # Calculate rank of y_hat
    rank = [(sum(D_hat[i][0] * mu_q) - D_hat[i][1]).detach().numpy() for i in range(len(D_hat))]
    y_hat_rank = rank[-1]
    
    # Add to conformal set if in not in bottom 10 percent of probabilities
    if np.count_nonzero(y_hat_rank > rank) / len(rank) > 0.1:
        conformal_set.append(copy.deepcopy(y_hat))
    else:
        print(f"{y_hat} Not added")
        
    y_hat -= step_size
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

NameError: name 'copy' is not defined

In [102]:
print(conformal_set)
print(f"Y[-1]: {Y[-1]}")
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")

[tensor(-0.4545, dtype=torch.float64), tensor(0.0455, dtype=torch.float64)]
Y[-1]: 0.014277702645567266
Y[-1] is covered
Elapsed Time: 0.8137557506561279
